<a href="https://colab.research.google.com/github/beyza720/CENG463-Assignment2/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install transformers datasets torch
!pip install evaluate

In [56]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from google.colab import drive

drive.mount('/content/drive')


train_path = '/content/drive/My Drive/Colab Notebooks/power-lv-train.tsv'

train_power_data = pd.read_csv(train_path, sep='\t')


# Class imbalance for power
class_0 = train_power_data[train_power_data['label'] == 0]
class_1 = train_power_data[train_power_data['label'] == 1]

class_1_oversampled = resample(class_1, replace=True, n_samples=len(class_0), random_state=42)

balanced_train_power_data = pd.concat([class_1_oversampled, class_0])


# split the train data into 0.9 train and 0.1 validation
train_power, val_power = train_test_split(
    balanced_train_power_data,
    test_size=0.1,
    stratify=balanced_train_power_data['label'],
    random_state=42
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-cased", num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_power)
val_dataset = Dataset.from_pandas(val_power)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

In [59]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [60]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=107,
    save_steps=107,
    save_total_limit=3
)

In [61]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [62]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [63]:
from transformers import Trainer

In [64]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

In [65]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.677600,0.651975,0.592593,0.596935,0.592593,0.588656
2,0.627800,0.601310,0.687831,0.718264,0.687831,0.677083
3,0.496600,0.506065,0.777778,0.778326,0.777778,0.777703


TrainOutput(global_step=321, training_loss=0.6006611856716073, metrics={'train_runtime': 128.1541, 'train_samples_per_second': 39.772, 'train_steps_per_second': 2.505, 'total_flos': 1341077049169920.0, 'train_loss': 0.6006611856716073, 'epoch': 3.0})

In [66]:
# the following part is for the casual language model, I fine-tune this model also  to get better results

In [67]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
train_encodings = tokenizer(list(train_power['text']), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_power['text']), truncation=True, padding=True, max_length=512)

In [69]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = Dataset(train_encodings, train_power['label'].tolist())
val_dataset = Dataset(val_encodings, val_power['label'].tolist())

In [70]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # save_strategy, evaluation_strategy ile aynı olmalı
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [71]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-71-f2a888a7213a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [72]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.679700,0.685878
2,0.658400,0.627027
3,0.622100,0.561853


TrainOutput(global_step=321, training_loss=0.6579410314931305, metrics={'train_runtime': 145.8307, 'train_samples_per_second': 34.951, 'train_steps_per_second': 2.201, 'total_flos': 1341077049169920.0, 'train_loss': 0.6579410314931305, 'epoch': 3.0})

In [73]:
val_preds = trainer.predict(val_dataset)
predicted_labels = torch.argmax(torch.tensor(val_preds.predictions), axis=1).numpy()
true_labels = val_power['label'].tolist()

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

import pandas as pd
performance_table = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [accuracy, precision, recall, f1]
})

print("\nModel: XLM-RoBERTa Base")
print(performance_table.to_string(index=False))

print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


Model: XLM-RoBERTa Base
   Metric    Value
 Accuracy 0.724868
Precision 0.725686
   Recall 0.724868
 F1 Score 0.724544

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.76      0.73        95
           1       0.74      0.69      0.71        94

    accuracy                           0.72       189
   macro avg       0.73      0.72      0.72       189
weighted avg       0.73      0.72      0.72       189

